# 特徴量選択
Wrapper_Sequence Forward Generation

In [70]:
# 必要なライブラリを読み込む
import datetime
from dateutil.relativedelta import relativedelta
import pickle

import pandas as pd
import numpy as np
import xgboost as xgb
# https://qiita.com/FJyusk56/items/0649f4362587261bd57a

import category_encoders as ce

import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# 必要な関数を読み込む
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # 精度の確認 / 正解率
from sklearn.metrics import f1_score       # f1

# 次元削減
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# User defined functions
from self_lib import tips
from self_lib import doggie_tail as d_
from self_lib import giraffe as graph
from self_lib import timer as t

# Jupyter上で図を表示するためのコマンド
%matplotlib inline

In [71]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs

In [72]:
with open('cleaned_data.pickle','rb') as f:
    data = pickle.load(f)

In [73]:
data.head()

,LoanStatus,Term,Approval_YM,Disbursement_YM,TermFlg,LoanEnd,ApprovalDate,ApprovalFY,DisbursementDate,RevLineCr,...,Employees,ExistNew,CreateJob,FranchiseCode,RetainedJob,Zip,BankState,UrbanRural,City,State
0,0,84,1996.09,1996.12,0,2003.09,23-Sep-96,1996,31-Dec-96,0,...,6,0.00,0,0,0,75601,0,0,LONGVIEW,TX
1,0,26,2004.02,2004.03,1,2006.04,24-Feb-04,2004,31-Mar-04,2,...,1,0.00,0,0,1,83814,0,1,COEUR D'ALENE,ID
2,0,180,1995.04,1995.07,0,2010.04,21-Apr-95,1995,31-Jul-95,1,...,2,0.00,0,0,0,59868,0,0,SEELEY LAKE,MT
3,0,84,2004.07,2004.08,0,2011.07,29-Jul-04,2004,31-Aug-04,0,...,6,0.00,0,0,7,3055,0,1,MILFORD (CENSUS NAME FOR MILFO,NH
4,0,60,2005.08,2005.11,0,2010.08,25-Aug-05,2005,30-Nov-05,0,...,1,0.00,0,0,0,39429,0,2,COLUMBIA,MS


In [74]:
# 学習データとテストデータの設定・分割
y=data['LoanStatus']

X=data.drop('LoanStatus',axis=1)
feature_cnt=len(X.columns) #特徴量の数
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [89]:
X_train.columns

Index(['Term', 'Approval_YM', 'Disbursement_YM', 'TermFlg', 'LoanEnd',
       'ApprovalDate', 'ApprovalFY', 'DisbursementDate', 'RevLineCr',
       'LowLoan', 'LoanAmount', 'BalanceGross', 'DisbursementGross',
       'GuaranteedLoan', 'SystemCode', 'Employees', 'ExistNew', 'CreateJob',
       'FranchiseCode', 'RetainedJob', 'Zip', 'BankState', 'UrbanRural',
       'City', 'State'],
      dtype='object')

In [87]:
y_train.name

'LoanStatus'

In [81]:
feature_cnt

25

In [92]:
# SFSインスタンス化
# feature_cnt=33  
sfs=SFS(LogisticRegression(),
       k_features=(1,feature_cnt),
       forward=True,
       floating=False,
       verbose=2,
       scoring='f1',
        cv=5,
        n_jobs=-1)

sfs=sfs.fit(X_train,y_train,custom_feature_names=X_train.columns)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   7 out of  25 | elapsed:    1.6s remaining:    4.2s
[Parallel(n_jobs=-1)]: Done  20 out of  25 | elapsed:    2.9s remaining:    0.6s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:    3.5s finished

[2021-01-17 01:45:10] Features: 1/25 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  24 | elapsed:    0.4s remaining:    1.3s
[Parallel(n_jobs=-1)]: Done  19 out of  24 | elapsed:    1.0s remaining:    0.2s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:    1.3s finished

[2021-01-17 01:45:11] Features: 2/25 -- score: nan[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  23 | elapsed:    0.3s remaining:    2.0s
[Parallel(n_jobs=-1)]: Done  16 out of  23 | elapsed:    1.1s remaining:    0.4s
[Parallel(n_jobs=-1)]: Done  23 out of 

KeyError: None

In [84]:
# 視覚化
fig=plot_sfs(sfs.get_metric_dict(),kind="std_dev",figsize=(14,10))
plt.title('Sequence Forward Generation (w. StdDev)')
plt.grid()

AttributeError: SequentialFeatureSelector has not been fitted, yet.

In [ ]:
features_sel_wrapper_sfg=list(sfs.k_features_names_)
ref=LogisticRegression().fit(X_train[features_sel_wrapper_sfg],y_train)
valid_result=f1_score(y_test,
                      reg.predict(X_train[features_sel_wrapper_sfg]))
score_sel_wrapper_sfg=f1_score(y_test,reg.predict(X_test[features_sel_wrapper_sfg]))
print(f'特徴量:{features_sel_wrapper_sfg}')
print(f'訓練:{train_result}')
print(score_sel_wrapper_sfg)